# Clase 6: Clustering

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**


## Objetivos de la Clase


- Comprender cuál es la utilidad de las técnicas de clustering.
- Analizar diversos tipos de algoritmos de clustering.

## Clustering

Clustering es la tarea que consiste en agrupar observaciones similares en grupos llamados *clusters*. La idea es que los grupos solo contengan información similar.
Es una tarea usual al realizar Análisis Exploratorio de Datos (EDA), ya que permite encontrar de forma automatizada grupos de observaciones similares.

Ya que no es necesario que el dataset esté etiquetado, es una técnica de aprendizaje no-supervisado.

<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/clustering.png?raw=true' width=800/>
</div>

<div align='center'>
    <span>Ejemplo de Clustering. Fuente: <a href='https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html'>Comparación de Clustering en Scikit-Learn.</a></span>
</div>

## Tipos de Clustering

Existen varias técnicas de clustering, las cuales se pueden clasificar en las siguientes categorías:


|  | **Particional** | **Jerárquico** | **Difuso** |
|---|---|---|---|
| Descripción | Divide los datos en clusters sin traslape, tal que cada dato está en un solo grupo y en ningún otro. | Agrupa ejemplos al ir estableciendo jerarquías entre estos, de tal manera que los datos son organizados como un árbol. | Cada objeto pertenece a cada cluster con un peso de pertenencia entre 0 y 1. |
| Ejemplos | K-Means, DBScan | Aglomerativo, Divisivo | Mixtura de Gaussianas |

## Clustering en `Scikit-learn`

Scikit-learn ofrece una gran gama de algoritmos de clustering para explorar.





<div align='center'>
    <img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/cluster_comparison.png?raw=true' width=800/>
</div>

<div align='center'>
    <span>Ejemplo de los distintos métodos de Clustering ofrecidos por Scikit-Learn. </span>
    <br>
    Fuente: <a href='https://scikit-learn.org/stable/modules/clustering.html'>Clustering en Scikit-Learn.</a>
</div>

---

## Problema de Hoy: 🎸🤘 Caracterización Musical 🎼🎵 

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/spotify.png?raw=true' width=200/>
</div>

 
    
Los atributos son: 

- `key`: escala de la canción. 0 = C, 1 = C♯/D♭, 2 = D...  [Mas información](https://en.wikipedia.org/wiki/Pitch_class).
- `modo`: 1 si la escala es mayor, 0 si es menor.
- `time_signature`: cuántos pulsos hay en cada compás. (4, 3,...).
- `loudness`: Volumen de la canción (rango -60, 0).


- `acousticness`: Probabilidad de que la canción sea solo acústica.
- `danceability`: Describe que tan bailable es la canción. (rango 0, 1).
- `energy`: Mide que tan energética es una canción (rango 0, 1).
- `instrumentalness`: Probabilidad que la canción contenga voces.
- `liveness`: Probabilidad de que la canción fuese grabada en vivo.
- `speechiness`: Probabilidad de que la canción sea exclusivamente vocal (ejemplo: podcast : 1). 
- `valence`: Sentimiento de la canción (rango 0, 1). 1 -> felicidad, alegria, euforia. 0 -> Tristeza, enojo, depresión.
- `tempo` : Pulsos por minuto de la canción (BPM). 
- `genre`: Género de la canción.



### Análisis Exploratorio de Datos

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../../recursos/2023-01/17-Clustering/descriptores_musica.csv")
df.head(5)

In [ ]:
df['genre'].unique()

In [ ]:
df.describe()

In [ ]:
def get_ejemplo(idx):
    """
    Obtiene un ejemplo y lo formatea como columna.
    """
    ejemplo = (
        df.loc[
            idx,
            [
                "danceability",
                "energy",
                "speechiness",
                "acousticness",
                "instrumentalness",
                "valence",
                "name",
                "artist",
                "genre",
            ],
        ]
        .to_frame()
        .reset_index()
    )
    ejemplo.columns = ["Descriptor", "Valor"]
    return ejemplo

In [ ]:
# pueden cambiar el índice de alguno de estos ejemplos para
# mostrar otra canción en la visualización
ejemplo1 = get_ejemplo(483)
ejemplo2 = get_ejemplo(166)
ejemplo3 = get_ejemplo(15)
ejemplo4 = get_ejemplo(810)

ejemplos = [ejemplo1, ejemplo2, ejemplo3, ejemplo4]

#### Spider/Radar Chart

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "polar"}, {"type": "polar"}],
        [{"type": "polar"}, {"type": "polar"}],
    ],
    subplot_titles=[
        ejemplo1.loc[6, "Valor"], ejemplo3.loc[6, "Valor"],
        ejemplo2.loc[6, "Valor"], ejemplo4.loc[6, "Valor"],
    ],
)

for i, ejemplo in enumerate(ejemplos):
    fig.add_trace(
        go.Scatterpolar(
            r=ejemplo.loc[0:5, "Valor"],
            theta=ejemplo.loc[0:5, "Descriptor"],
            fill="toself",
            name=f"{ejemplo.loc[6, 'Valor']} - {ejemplo.loc[7, 'Valor']} ({ejemplo.loc[8, 'Valor']})",
        ),
        col=i // 2 + 1,
        row=i % 2 + 1,
    )

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=False,
    title="ScatterPolar/Radar/Spider Chart/ Descripción de Ejemplos",
    height=700,
)

fig.show()

#### Histogramas

In [ ]:
px.histogram(df, x="duration_ms")

In [ ]:
px.histogram(df, x="loudness")

In [ ]:
px.histogram(df, x="tempo")

In [ ]:
dt_to_hists = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "liveness",
        "genre",
    ],
].melt(id_vars=["genre"], var_name="variable", value_name="valor")

px.histogram(
    dt_to_hists, x="valor", color="variable", facet_col="variable", facet_col_wrap=4
).update_layout(showlegend=False)

### Correlaciones

In [ ]:
corr = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",    ],
].corr()
px.imshow(corr)

---

## Preparando los Datos

Para este clase usaremos los siguientes atributos:

In [ ]:
df_ = df.loc[
    :,
    [
        "danceability",
        "energy",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "valence",
        "liveness",
        "duration_ms",
        "loudness",
        "tempo",
        "name",
        "genre",
    ],
]

df_

In [ ]:
df_.describe()

### `MinMaxScaler`

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

ct = ColumnTransformer(
  transformers = [
		(
			"MinMax", 
			MinMaxScaler(),
			["duration_ms", "loudness", "tempo"]
		)
	], 
  remainder="passthrough"
)


pipe = Pipeline(
	steps = [
		("Preprocesamiento", ct)
	]
)

In [ ]:
features_to_scale = df_.iloc[:, :-2]  # eliminar nombre y género

In [ ]:
# transformamos el resultado de la transformación a un dataframe:
scaled_features = pd.DataFrame(
  pipe.fit_transform(features_to_scale), 
  columns=features_to_scale.columns
)

scaled_features

In [ ]:
scaled_features.describe()

---

## `UMAP` - Proyectamos con UMAP

In [ ]:
from umap import UMAP

proyector = UMAP(
    random_state=88,
    n_neighbors=20,
    min_dist=0.15,
    n_components=2
)

projections = proyector.fit_transform(scaled_features)

In [ ]:
projections.shape

In [ ]:
# este dataframe lo estaremos usando para graficar de aquí en adelante
fig_df = pd.concat(
    [
        df.loc[:, ["name", "artist", "genre"]],
        scaled_features,
        pd.DataFrame(projections, columns=["x", "y"]),
    ],
    axis=1,
)

fig_df

In [ ]:
def get_scatter(fig_df, color_col):
    fig = px.scatter(
        fig_df,
        x="x",
        y="y",
        color=color_col,
        hover_name=df["artist"] + " - " + df["name"],
        # labels={"genre": "Género Musical"},
        hover_data=[
            "danceability",
            "energy",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "valence",
            'duration_ms',
            'loudness',
            'tempo',
            'genre'
        ],
        range_x=(fig_df["x"].min() - 1, fig_df["x"].max() + 1),
        range_y=(fig_df["y"].min() - 1, fig_df["y"].max() + 1),
    )
    return fig


get_scatter(fig_df, "genre")

---

**Aquí empezamos clustering**

### K-Means

Técnica de clustering de tipo particional.

Encuentra centros de clusters que minimizan la suma de distancias entre los datos y el centro de cada cluster.

Algoritmo:

---


    Seleccionar K centroides iniciales.
    repite: 
        Asigna todos los puntos a sus centros más cercanos.
        Recomputa los centros de cada cluster.
    hasta que los centros no cambien.

---
*Hasta que los centros no cambian* equivale a minimizar la suma de errores cuadrados SSE:

$$SSE = \sum_{i=1}^{K} \sum_{x\in C_i} d(c_i, x)^2 $$

Comunmente la función de distancia $d$ es la distancia euclideana $d(x,y) = \sqrt{\sum_{i}(x_i, y_i)^2}$ .

<div align='center'>

<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/kmeans_example.png?raw=true' width=500/>
</div>


> **Pregunta**: ¿Qué sucede si se eligen mal los centroides iniciales?

In [ ]:
# Random State permite controlar la aleatoridad.
# Es decir, permite generar los mismos números aleatorios en distintas ejecuciones.
RANDOM_STATE = 99

In [ ]:
from sklearn.cluster import KMeans

# El número de clusters es parámetro. En este caso, es 2.
kmeans = (
 KMeans(
    n_clusters=2, 
    random_state=RANDOM_STATE,
    n_init=10,
 )
 .fit(scaled_features)
)

labels = kmeans.labels_

Podemos acceder a los centroides calculados.

In [ ]:
# Clusters calculados por cada observación de entrenamiento
labels

In [ ]:
# Centroides calculados.
kmeans.cluster_centers_

In [ ]:
# También podemos acceder a la suma de errores cuadráticos (llamada inercia en scikit)
kmeans.inertia_

In [ ]:
# Cuantas features utilizó
kmeans.n_features_in_

In [ ]:
fig_df["kmeans_labels_2"] = kmeans.labels_
fig = get_scatter(fig_df, "kmeans_labels_2")

# Transformamos los centro que vimos anteriormentes a la proyección 2d.
projected_centers = proyector.transform(kmeans.cluster_centers_)

fig.add_trace(
    go.Scatter(
        x=projected_centers[:, 0],
        y=projected_centers[:, 1],
        mode="markers",
        # name="Centros",
        marker_size=12,
        marker_color="LightSlateGray",
        showlegend=False,
    )
)

In [ ]:
kmeans = (
    KMeans(
        n_clusters=5,
        random_state=RANDOM_STATE, 
        n_init='auto'
		).fit(scaled_features)
)
labels = kmeans.labels_


fig_df["kmeans_labels_5"] = kmeans.labels_
fig = get_scatter(fig_df, "kmeans_labels_5")

In [ ]:
# Transformamos los centro que vimos anteriormentes a la proyección 2d.
centros_pryectados = proyector.transform(kmeans.cluster_centers_)

fig = get_scatter(fig_df, "kmeans_labels_5")
fig.add_trace(
    go.Scatter(
        x=centros_pryectados[:, 0],
        y=centros_pryectados[:, 1],
        mode="markers",
        name="Centros",
        marker_size=12,
        marker_color="red",
        showlegend=False,
    )
)

> **Pregunta:** ¿Cómo identificamos la cantidad de cluster óptimos?


### Método del Codo



![Método del Codo](https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/elbow.png?raw=true)

In [ ]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=0, n_init='auto').fit(scaled_features).inertia_]
    for i in range(2, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

In [ ]:
px.line(
    intertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

**Alternativa: Coeficiente de Silohuette**


El valor de la silueta es una medida de cuán similar es un objeto respecto a su propio cluster en comparación con los otros.  


Por cada elemento se calcula: 

$$s = \frac{b-a}{\max(a,b)}$$

donde: 

- $a$ es el promedio de las distancias entre el elemento y todos los otros elementos del cluster al cual el elemento analizado fue etiquetado.
- $b$ es la distancia promedio entre el elemento y todos los otros elementos del cluster más cercano.

La silueta va de -1 a +1 donde 
- un valor cercano a 1: indica que el objeto está bien clusterizado
- cercano a 0: indica que el elemento está entre dos clusters
- cercano a $-1$: indica que el elemento está mal asignado. 

Si la mayoría de los objetos tienen un valor alto, entonces podemos decir que los elementos del cluster están bien asignados.

Por último, el coeficiente de silueta se calcula al promediar todos los coeficientes individuales.

In [ ]:
from sklearn.metrics import silhouette_score

scores = [
    [
        i,
        silhouette_score(
            scaled_features,
            KMeans(n_clusters=i, random_state=RANDOM_STATE, n_init='auto').fit(scaled_features).labels_,
        ),
    ]
    for i in range(2, 20)
]

scores = pd.DataFrame(scores, columns=["n° clusters", "silhouette_score"])
scores.head(10)

In [ ]:
px.line(scores, x="n° clusters", y="silhouette_score")

**Más opciones:**

https://medium.com/@haataa/how-to-measure-clustering-performances-when-there-are-no-ground-truth-db027e9a871c

> **Pregunta ❓**: ¿Podría usar k-means con datos categóricos?

En general, los algoritmos de clustering tradicionales están diseñados para trabajar con datos numéricos y utilizan medidas de distancia, como la euclidiana, para determinar la similitud entre los puntos de datos. Sin embargo, aplicar estas técnicas directamente a datos categóricos puede ser inapropiado, ya que la distancia euclidiana pierde su significado cuando se trata de atributos que no son numéricos. Esto puede llevar a interpretaciones erróneas y resultados de clustering no fiables. Es crucial seleccionar algoritmos, como k-modes, que estén específicamente adaptados para manejar datos categóricos y que utilicen métricas de disimilitud adecuadas para este tipo de información.

<div align='center'>
<img src=../../recursos/2025-01/imagenes_clase_6/distancia_euclidiana.png width=300 />
</div>

Una forma de solucionar este problema es utilizando [kmodes](https://pypi.org/project/kmodes). Esto se da, gracias a que el algoritmo de k-modes maneja eficientemente los datos categóricos mediante el uso de una medida de disimilitud de coincidencia simple, la que evalúa cuán diferentes son los puntos de datos basándose en las categorías que no comparten. A diferencia del algoritmo k-means, que calcula los centros de los clústeres promediando, k-modes determina los centros de los clústeres usando modas, es decir, las categorías más frecuentes en el clúster. Durante el proceso de agrupamiento, k-modes emplea un método basado en la frecuencia para actualizar estas modas. Este enfoque tiene como objetivo minimizar la función de costo de agrupamiento, mejorando la efectividad y precisión general del agrupamiento.

<div align='center'>
<img src=../../recursos/2025-01/imagenes_clase_6/kmode.png width=800 />
</div>

---


## `DBSCAN`
	

Algoritmo de clustering basado en densidad. Ideal para buscar outliers.

- Densidad: Número de puntos en un círculo.
- Idea: Regiones densas representan clusters.

Parámetros: 

- `Eps`: radio de los círculos
- `MinPts`: número mínimo de puntos de una región.


- Es comunmente resistente al ruido.
- Problemas en regiones con distintas densidades.

Tipos de puntos:
- **Punto núcleo**: tiene al menos `min_samples` dentro de su `eps`. Es considerado un punto denso.
- **Punto frontera**: tiene menos de `min_samples` dentro de su vecindario, pero está cerca de un núcleo. Está conectado, pero no forma parte del núcleo.
- **Punto ruido**: no es núcleo ni frontera, por lo tanto se considera un punto aislado (outlier).


<div align='center'>
<img src=../../recursos/2025-01/6-Clustering/dbscan.png width=800 />
</div>

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.3, min_samples=3)
dbscan.fit(scaled_features)

fig_df["dbscan"] = pd.Series(dbscan.labels_, dtype=str)

get_scatter(fig_df, "dbscan")

### Visualizando el parámetro eps

In [ ]:
import numpy as np

epss = np.arange(0.1, 1, 0.2)
epss

In [ ]:
clustering = [
    DBSCAN(eps=eps, min_samples=2).fit_predict(scaled_features) for eps in epss
]

dbscan_labels = pd.DataFrame(np.array(clustering)).T
dbscan_labels.columns = np.round(epss, 3)

dbscan_labels["x"] = projections[:, 0]
dbscan_labels["y"] = projections[:, 1]

dbscan_labels = dbscan_labels.melt(
    id_vars=["x", "y"], var_name="eps", value_name="label"
)
dbscan_labels["label"] = dbscan_labels["label"].astype(str)
dbscan_labels.sample(10)

In [ ]:
fig = px.scatter(
    dbscan_labels,
    x="x",
    y="y",
    facet_row="eps",
    color="label",
    height=1600,
)
fig.show()

In [ ]:
dbscan_labels.loc[dbscan_labels['eps'] == 0.3, 'label'].value_counts()

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt

neighbors = NearestNeighbors(n_neighbors=2).fit(scaled_features)
distances, indices = neighbors.kneighbors(scaled_features)
distances = np.sort(distances[:, 1])
plt.plot(distances)
plt.title("Gráfico de k-distancias")
plt.xlabel("Puntos ordenados")
plt.ylabel("Distancia al 2º vecino")
plt.grid(True)
plt.show()

---

## Clustering Jerárquico
</br>
<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/clustering_jerarquico.png?raw=true' width=400 />
</div>

> **Pregunta**: ¿Cómo los agrupamos?

Requieren que exista una **definición de distancia** entre los elementos que se desean agrupar.

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/matriz_distancias.png?raw=true' width=800 />
</div>


### Tipos

#### Aglomerativo

- Empezar con cada punto como cluster individual.
- En cada paso, mezclar el par de clusters más cercano hasta que quede sólo un cluster (o k clusters).

#### Divisivo

- Empezar con un cluster que contenga todos los puntos
- En cada paso, dividir un cluster en dos hasta que todo cluster contenga un solo punto (o haya k clusters).




### Algortimo básico Aglomerativo


---

    Partimos con que cada punto es cluster por separado.
    Calculamos la matriz de distancias.
    Repetimos :
      Unimos los puntos/clusters (usando la matriz de distancias) en un solo cluster según algún criterio/enlace.
	  Se recalcula la matriz de distancia.
    hasta que ya no podamos unir nada más.
---


### Tipos de Enlaces entre Clusters


El enlace determina como se irán uniendo los distintos clusters que se irán generando. Existen varias opciones: 

- **Máx - Enlace Completo** (`complete`): Va uniendo puntos/clusters más lejanos.


- **Min - Enlace Simple** (`simple`): Va uniendo puntos/clusters más cercanos.


- **Promedio entre grupos** (`average`): Va uniendo según el promedio de la distancia de todos contra todos


- **Ward (`ward`)**: Va uniendo al minimizar la suma de las diferencias cuadradas entre cluster. Muy similar a lo que hace K-Means.


La opción elegida puede provocar variaciones gigantezcas entre los clusters producidos.

In [ ]:
import numpy as np
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import average, complete, single, ward
from sklearn.cluster import AgglomerativeClustering

#### Máx - Enlace Completo (`complete`)
    
<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_max.png?raw=true' width=400 />
</div>
    
$$\max\{(d(x,y): x \in A, y \in B)\}$$  
    
    
- Solo se agrupan dos clusters si todos sus puntos están suficientemente cerca entre sí.
- Produce clusters esféricos, bien separados, y menos sensibles a outliers.

In [ ]:
sample = scaled_features.sample(20, random_state=RANDOM_STATE)
sample

In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=complete,
    orientation="left",
    color_threshold=1.5,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="complete")
labels = model.fit_predict(scaled_features)

fig_df["ag_complete"] = pd.Series(model.labels_, dtype=str)

get_scatter(fig_df, "ag_complete")

In [ ]:
pd.Series(model.labels_, dtype=str).value_counts()

#### Mín - Enlace Simple (`single`)

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_min.png?raw=true' width=400 />
</div>

$$\min\{(d(x,y): x \in A, y \in B)\}$$  


- Agrupa puntos uno por uno a lo largo del camino más corto.
- Resulta en clusters muy alargados.
- Es muy sensible al ruido o a “puentes” de puntos intermedios.



In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=single,
    orientation="left",
    color_threshold=0.4,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="single")
labels = model.fit_predict(scaled_features)


fig_df["ag_single"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_single")

In [ ]:
pd.Series(model.labels_, dtype=str).value_counts()

#### Promedio (`average`)

<div align='center'>
<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/17-Clustering/agglomerative_mean.png?raw=true' width=400 />
</div>


- Usa la distancia promedio entre todos los puntos de dos clusters.
- No es tan estricto como complete ni tan flexible como single.
- Produce resultados más balanceados, pero puede ser más lento.

In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=average,
    orientation="left",
    color_threshold=0.8,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="average")
labels = model.fit_predict(scaled_features)

fig_df["ag_average"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_average")

In [ ]:
pd.Series(model.labels_, dtype=str).value_counts()

#### Ward (`ward`)

$$Distancia(Ci,Cj)=SSE(Ci∪Cj)−SSE(Ci)−SSE(Cj)$$
    

Busca minimizar el incremento del SSE cuando se mezclan dos clusters


In [ ]:
# plot the top three levels of the dendrogram
ff.create_dendrogram(
    sample,
    labels=(
        df.loc[sample.index, "artist"] + " - " + df.loc[sample.index, "name"]
    ).values,
    linkagefun=ward,
    orientation="left",
    color_threshold=0.8,
).update_layout(width=1000, height=800)

In [ ]:
model = AgglomerativeClustering(n_clusters=5, linkage="ward")
labels = model.fit_predict(scaled_features)

fig_df["ag_ward"] = pd.Series(model.labels_, dtype=str)
get_scatter(fig_df, "ag_ward")

In [ ]:
pd.Series(model.labels_, dtype=str).value_counts()

### Resumen clustering jerárquicos

Ventajas: 
    
- No hay que suponer a priori el número de clases.
- Podemos escoger el número de clusters cortando el dendograma. 


Desventajas:

- No hay función objetivo por minimizar.
- Diferentes esquemas presentan diferentes problemas.

## Gaussian Mixture

<center>
<img src="../2024-01/imagenes_clase_12/gaussian-mixture-models-1.png" width=300 />
<center>
<img src="../2024-01/imagenes_clase_12/kmeans_vs_gmm_2.png" width=300 />

Un Modelo de Mezclas Gaussianas (GMM) es un modelo probabilístico que asume que todos los datos generados provienen de una combinación de varias distribuciones gaussianas (normales) con parámetros desconocidos. Es utilizado ampliamente en tareas de clustering debido a su capacidad para modelar clústeres que tienen formas elipsoidales y diferentes tamaños. 

> En esencia, un GMM intenta estimar las medias y las covarianzas de las distribuciones gaussianas que mejor describen los datos, y asigna probabilidades de pertenencia a cada clúster para cada punto del dataset.

Entonces un GMM modela la función de densidad conjunta como una suma ponderada de $K$ gaussianas multivariadas:

$$p(x)= \sum_{k=1}^K \pi_k \cdot \mathcal{N}(x \mid \mu_k, \Sigma_k)$$

Cada componente en el GMM tiene tres parámetros principales:

- Media (μ): El centro del clúster.
- Covarianza (Σ): Define la forma y orientación del clúster.
- Peso (π): el peso de un cluster.

Algoritmo Expectation-Maximization (EM)

Para ajustar un GMM a los datos, se utiliza el algoritmo Expectation-Maximization (EM), que es un enfoque iterativo para encontrar estimadores de máxima verosimilitud de parámetros en modelos estadísticos dependientes de variables latentes. El EM alterna entre dos pasos:

- Expectation (E): Calcula la probabilidad de que cada observación pertenezca a cada clúster, basado en los parámetros actuales de las distribuciones gaussianas.
- Maximization (M): Actualiza los parámetros de las distribuciones (medias, covarianzas, pesos) para maximizar la verosimilitud de los datos dados las probabilidades calculadas en el paso E.

### Funcionamiento resumido

<center>
<img src="../../recursos/2025-01/imagenes_clase_6/gmm_gif.gif" width=300 />

1. **Inicializar Parámetros**: Inicializa las medias $\mu_k$, las matrices de covarianza $\Sigma_k$ y los coeficientes de mezcla $\pi_k$ con valores aleatorios o predefinidos.
2. **Calcular probabilidad de pertenencia a cada cluster**: Calcula la probabilidad de que una observación provenga de un cluster $k$.
3. **Reestimar Parámetros**: Actualiza todos los parámetros utilizando las asignaciones encontradas en el paso 2.
4. **Calcular la Log-verosimilitud**: Calcula la log-verosimilitud de los datos dados el modelo.
5. **Verificar Convergencia**: Define un criterio de convergencia. Si el valor de la log-verosimilitud se estabiliza (o si todos los parámetros convergen), detén el proceso. De lo contrario, regresa al Paso 2.

In [ ]:
from sklearn.mixture import GaussianMixture 

gmm = GaussianMixture(n_components = 5, random_state=32) 
gmm.fit(scaled_features)

In [ ]:
# Para obtener las labels
gmm.predict(scaled_features)

In [ ]:
# Para obtener las probabilidaes
gmm.predict_proba(scaled_features)

In [ ]:
#caso particular
gmm.predict_proba(scaled_features)[0]

In [ ]:
# Obtener los centroides de las distribuciones
gmm.means_

In [ ]:
# número de cluster
gmm.n_components

In [ ]:
# pesos
print(gmm.weights_)

In [ ]:
gmm.weights_.sum()

In [ ]:
fig_df["gmm_labels"] = gmm.predict(scaled_features)
fig_gmm = get_scatter(fig_df, "gmm_labels")

# Transformamos los centro que vimos anteriormentes a la proyección 2d.
projected_centers_gmm = proyector.transform(gmm.means_)

fig_gmm.add_trace(
    go.Scatter(
        x=projected_centers_gmm[:, 0],
        y=projected_centers_gmm[:, 1],
        mode="markers",
        marker_size=12,
        marker_color="LightSlateGray",
    )
)

### En síntesis GMM vs KMeans

<center>
<img src="../2024-01/imagenes_clase_12/procons.gif" width=300 />

**Pros**

<center>
<img src="../2024-01/imagenes_clase_12/gmm_final.png" width=300 />

1. **Formas de los clústeres**: K-means asume que los clústeres son esféricos y tienden a tener un tamaño similar en términos de número de puntos de datos. Por lo tanto, funciona mejor cuando esta suposición se mantiene. En cambio, GMM es más flexible ya que puede acomodar clústeres que tienen formas elipsoidales y diferentes tamaños. Esto es posible porque GMM modela cada clúster usando una distribución gaussiana.

2. **Asignación suave**: K-means asigna cada punto de datos a un solo clúster (asignación dura), lo que puede ser limitante cuando un punto se sitúa cerca del límite entre dos clústeres. GMM, por otro lado, asigna probabilidades a cada punto de datos para pertenecer a múltiples clústeres (asignación suave). Esto proporciona una imagen más matizada de la estructura de los datos.

3. **Robustez en la distribución de los datos**: GMM puede ser más efectivo que K-means en situaciones donde los datos no están uniformemente distribuidos entre los clústeres o cuando hay una mezcla compleja en la distribución de los datos.

**Contras**

1. Sensibilidad a los valores iniciales: Los resultados del EM pueden variar significativamente dependiendo de los valores iniciales.
2. Optimización local: GMM puede quedar atrapado en óptimos locales, especialmente si el espacio de datos es complejo o si hay muchos parámetros a estimar.
3. Escalabilidad: GMM puede ser computacionalmente costoso con un gran número de componentes o un gran volumen de datos.